### Notebook for classsifying using a Artificial Neural Network (NN).

This notebook is excpecting the data to be numeric. A method of ensuring this to first use Pre_Processing_USE-4.ipynb.

 Cells are executed in the order in which they appear in the notebook.

In [1]:
#%load_ext tensorboard # Uncomment this line to use tensorboard

# Import libraries
import tensorflow as tf
from keras.models import Sequential
from keras.losses import BinaryCrossentropy
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import datetime

import keras_tuner as kt

Import numerical feature and label data.

In [2]:
# Load dataset
X = pd.read_csv('X_numeric.csv', sep=',', header=None)
y = pd.read_csv('y_numeric.csv', sep=',', header=None)
y = np.ravel(y)

In [3]:
# Check the data in case the first row is the index 
X.head()

,0,1,2,3,4,5,6,7,8,9,...,61,62,63,64,65,66,67,68,69,70
0,1.480624e+10,1.480624e+10,1.480624e+10,16.178530,12.164207,1.753315,-5.300097,1.016841,8.118670,6.819417,...,11.372132,7.036551,-2.552680,-54.153320,32.803783,16.710754,-24.062065,6.590991,-0.729850,6.655100
1,4.475470e+11,4.475470e+11,4.475470e+11,20.004223,11.334753,-2.820019,-7.355494,-1.181901,6.805092,0.625156,...,21.825056,13.261285,-17.120144,4.034127,-10.328978,-12.635922,-22.477568,3.983698,-0.710531,4.778454
2,4.536947e+09,4.536947e+09,4.536947e+09,19.789394,11.128526,-5.510750,-8.971125,4.214148,-21.717592,6.829502,...,-4.366057,6.523605,-5.956306,8.049953,-8.431773,-1.328702,19.821604,2.230384,-2.804008,4.660541
3,3.897625e+10,3.897625e+10,3.897625e+10,19.559141,11.153430,-7.129503,14.209820,4.214219,-21.717434,-10.894494,...,12.437857,30.452587,15.085895,-17.211315,-9.998230,-9.034165,21.270443,-3.827162,-2.322451,4.214811
4,1.425274e+10,1.425274e+10,1.425274e+10,19.156839,11.093230,-4.028421,3.910608,0.271392,7.674269,3.828856,...,12.387394,15.096603,4.221771,-12.505415,6.967276,-8.013271,13.346799,3.252636,1.021878,7.317034


In [4]:
# Check the shape of the labels in case the first row is the index
y.shape

(657,)

In [ ]:
# If the index has been added to the first column, run this cell 
X=X.iloc[:,1:]
y=y.iloc[:,1:]

Split to train and test sets. Save them as np.array and make sure they are float. Perform scaling.

In [5]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
# Save to array and make sure dtype is float or else the scaling might not work.
X_train=np.asarray(X_train).astype(np.float_)
X_test=np.asarray(X_test).astype(np.float_)

y_train=np.asarray(y_train).astype(np.float_)
y_test=np.asarray(y_test).astype(np.float_)

In [7]:
# Scale the features
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

Design the model using Keras Tuner, find optimal learning rate and epochs. Train the model and check the accuracy.

In [8]:
# Create the model using Keras Tuner.
def model_builder(hp):
  model = Sequential()

  # Tune the number of units up to five Dense layers
  # Choose an optimal value of units between 10-100
  for i in range(hp.Int('layers', 0, 5)):
    model.add(Dense(units=hp.Int('units_' + str(i), 10, 100, step=10), activation=hp.Choice('act_' + str(i), ['relu', 'sigmoid', 'tanh'])))
  model.add(Dense(1, activation='sigmoid'))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=BinaryCrossentropy(from_logits=False),
                metrics=['accuracy'])

  return model

model_builder(kt.HyperParameters())

In [9]:
# Create the tuner
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=20,
                     hyperband_iterations=10,
                     factor=3,
                     directory='my_dir',
                     project_name='hyperpartisan_news_classifier_statuses6')

INFO:tensorflow:Reloading Oracle from existing project my_dir\hyperpartisan_news_classifier_statuses6\oracle.json
INFO:tensorflow:Reloading Tuner from my_dir\hyperpartisan_news_classifier_statuses6\tuner0.json


In [10]:
# Stop the tuner if validation loss does not improve for 5 consecutive epochs
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [11]:
# Run the tuner
tuner.search(X_train, y_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=3)[0]

INFO:tensorflow:Oracle triggered exit


In [12]:
# Get the summary of the best model
tuner.results_summary(1)

Results summary
Results in my_dir\hyperpartisan_news_classifier_statuses6
Showing 1 best trials
Trial summary
Hyperparameters:
layers: 3
learning_rate: 0.0001
units_0: 80
act_0: tanh
units_1: 80
act_1: relu
units_2: 80
act_2: tanh
units_3: 40
act_3: relu
units_4: 80
act_4: relu
tuner/epochs: 7
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 0.7931249737739563


In [13]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs to check the optimal number of epochs,
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train, y_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))


Epoch 1/50
14/14 [==============================] - 1s 22ms/step - loss: 0.5862 - accuracy: 0.7214 - val_loss: 0.5522 - val_accuracy: 0.7619
Epoch 2/50
14/14 [==============================] - 0s 5ms/step - loss: 0.5667 - accuracy: 0.7357 - val_loss: 0.5434 - val_accuracy: 0.7429
Epoch 3/50
14/14 [==============================] - 0s 5ms/step - loss: 0.5527 - accuracy: 0.7357 - val_loss: 0.5400 - val_accuracy: 0.7429
Epoch 4/50
14/14 [==============================] - 0s 5ms/step - loss: 0.5427 - accuracy: 0.7429 - val_loss: 0.5393 - val_accuracy: 0.7429
Epoch 5/50
14/14 [==============================] - 0s 5ms/step - loss: 0.5344 - accuracy: 0.7381 - val_loss: 0.5409 - val_accuracy: 0.7333
Epoch 6/50
14/14 [==============================] - 0s 5ms/step - loss: 0.5272 - accuracy: 0.7429 - val_loss: 0.5398 - val_accuracy: 0.7333
Epoch 7/50
14/14 [==============================] - 0s 5ms/step - loss: 0.5207 - accuracy: 0.7429 - val_loss: 0.5409 - val_accuracy: 0.7333
Epoch 8/50
14/14 [=

In [14]:
# Build the model with the optimal hyperparameters from the tuner
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model with the optimal number of epochs and log the results to tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
hypermodel.fit(X_train, y_train, epochs=best_epoch, validation_split=0.2, callbacks=[tensorboard_callback])


Epoch 1/20
14/14 [==============================] - 1s 31ms/step - loss: 0.6519 - accuracy: 0.6095 - val_loss: 0.6077 - val_accuracy: 0.7238
Epoch 2/20
14/14 [==============================] - 0s 13ms/step - loss: 0.6105 - accuracy: 0.6905 - val_loss: 0.5911 - val_accuracy: 0.7143
Epoch 3/20
14/14 [==============================] - 0s 12ms/step - loss: 0.5845 - accuracy: 0.7333 - val_loss: 0.5843 - val_accuracy: 0.7429
Epoch 4/20
14/14 [==============================] - 0s 14ms/step - loss: 0.5680 - accuracy: 0.7357 - val_loss: 0.5843 - val_accuracy: 0.7429
Epoch 5/20
14/14 [==============================] - 0s 14ms/step - loss: 0.5579 - accuracy: 0.7381 - val_loss: 0.5856 - val_accuracy: 0.7429
Epoch 6/20
14/14 [==============================] - 0s 14ms/step - loss: 0.5489 - accuracy: 0.7381 - val_loss: 0.5867 - val_accuracy: 0.7429
Epoch 7/20
14/14 [==============================] - 0s 24ms/step - loss: 0.5425 - accuracy: 0.7381 - val_loss: 0.5890 - val_accuracy: 0.7429
Epoch 8/20
14

In [15]:
# Evaluate the model on the test set
test_loss, test_accuracy = hypermodel.evaluate(X_test, y_test)
print(f'Results from test set: loss: {test_loss} - accuracy: {test_accuracy}')

5/5 [==============================] - 0s 3ms/step - loss: 0.5242 - accuracy: 0.7500
Results from test set: loss: 0.5242112874984741 - accuracy: 0.75


In [ ]:
# Check the results in tensorboard
%tensorboard --logdir logs/fit